<a href="https://colab.research.google.com/github/Anoziqq/NN/blob/main/Lab6/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторна робота 6
#З дисципліни "Нейронні мережі"
##Виконав студент:
###групи АнД-31
###Лаврій Руслан

## Завдання
## 1. Виконати вирішення задачі класифікації для 3 класів з набору даних food101 з використанням різних моделей нейронних мереж:
## 1.1 CNN модель з лабораторної роботи 4
## 1.2 Resnet модель
## 1.3 Efficientnet модель (моделі 1.1-1.3 з використанням оптимізатора Adam)
## 1.4 (1.5) Моделі 1.2,1.3 з використанням оптимізатора SGD.
## 1.6 (1.7) Моделі 1.2,1.3 отримані за допомогою tf.keras.applications та треновані з використанням fine-tuning (останні 10 шарів)
## 2. Індекси класів визначити індивідуально за залежностями: i1=n-1,i2=n+29,i3=n+59 (де і1,і2,і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n - номер за списком групи.
## 3. Порівняти результати моделювання із використанням TensorBoard
## 4. Графік(и) порівняння результатів завантажити у форматі .svg та вставити у підсумковий файл поряд із та відповідними висновками
## i1=5, i2=35, i3=65


In [9]:
import tarfile
import os
import shutil
import pathlib
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [4]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
file = tarfile.open("food-101.tar.gz")
file.extractall()
file.close()
!ls food-101

--2022-06-06 13:27:58--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2022-06-06 13:27:58--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  31.7MB/s    in 2m 57s  

2022-06-06 13:30:56 (26.9 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]

images	license_agreement.txt  meta  README.txt


## Знаходження класів

In [5]:
file = open('food-101/meta/classes.txt')
line_numbers = [5, 35, 65] 
lines = []
for i, line in enumerate(file):
  if i in line_numbers:
    lines.append(line.strip())
print(lines)

['beet_salad', 'escargots', 'mussels']


## Створення папок і копіювання фото

In [6]:
!mkdir food-101/test
!mkdir food-101/train
!mkdir food-101/test/beet_salad
!mkdir food-101/test/escargots
!mkdir food-101/test/mussels
!mkdir food-101/train/beet_salad
!mkdir food-101/train/escargots
!mkdir food-101/train/mussels

file = open('food-101/meta/train.txt')
for i in file:
  dir = i.split("/")
  if (dir[0] == 'beet_salad'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/train/beet_salad')
  elif(dir[0] == 'escargots'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/train/escargots')
  elif(dir[0] == 'mussels'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/train/mussels')

file = open('food-101/meta/test.txt')
for i in file:
  dir = i.split("/")
  if (dir[0] == 'beet_salad'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/test/beet_salad')
  elif(dir[0] == 'escargots'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/test/escargots')
  elif(dir[0] == 'mussels'):
    shutil.copy('food-101/images/'+i.replace('\n','') +'.jpg', 'food-101/test/mussels')
!ls food-101/test
test_dir = 'food-101/test'
train_dir = 'food-101/train'
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

beet_salad  escargots  mussels
['beet_salad' 'escargots' 'mussels']


## Перетворення зображень та покращення

In [7]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='categorical')


Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.
Found 2250 images belonging to 3 classes.


## CNN

In [8]:
model_1 = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(),
  Dense(128, activation='relu'),
  Conv2D(64, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3, activation='softmax')
])

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/cnn', histogram_freq=1)

model_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

history_1 = model_1.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks = [tb_callback])

Epoch 1/10
71/71 [==============================] - 50s 528ms/step - loss: 1.1738 - accuracy: 0.3711 - val_loss: 1.0283 - val_accuracy: 0.5013
Epoch 2/10
71/71 [==============================] - 35s 498ms/step - loss: 1.0260 - accuracy: 0.4587 - val_loss: 0.9578 - val_accuracy: 0.5240
Epoch 3/10
71/71 [==============================] - 35s 494ms/step - loss: 1.0028 - accuracy: 0.5000 - val_loss: 0.8773 - val_accuracy: 0.6173
Epoch 4/10
71/71 [==============================] - 37s 525ms/step - loss: 0.9660 - accuracy: 0.5658 - val_loss: 0.8133 - val_accuracy: 0.6800
Epoch 5/10
71/71 [==============================] - 35s 498ms/step - loss: 0.9268 - accuracy: 0.6084 - val_loss: 0.8871 - val_accuracy: 0.6627
Epoch 6/10
71/71 [==============================] - 35s 495ms/step - loss: 0.9350 - accuracy: 0.5773 - val_loss: 0.9217 - val_accuracy: 0.5720
Epoch 7/10
71/71 [==============================] - 36s 504ms/step - loss: 0.9330 - accuracy: 0.5684 - val_loss: 0.7774 - val_accuracy: 0.6493

## Resnet ADAM

In [10]:
model_2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_2.build([None, 224, 224, 3])

tb_callback_2 = tf.keras.callbacks.TensorBoard(log_dir='logs/resnet(adam)notunnig', histogram_freq=1)

model_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

history_2 = model_2.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_2])


Epoch 1/10
71/71 [==============================] - 56s 678ms/step - loss: 0.5805 - accuracy: 0.7720 - val_loss: 0.2696 - val_accuracy: 0.9120
Epoch 2/10
71/71 [==============================] - 46s 643ms/step - loss: 0.3298 - accuracy: 0.8787 - val_loss: 0.2225 - val_accuracy: 0.9253
Epoch 3/10
71/71 [==============================] - 44s 626ms/step - loss: 0.2670 - accuracy: 0.9049 - val_loss: 0.1955 - val_accuracy: 0.9347
Epoch 4/10
71/71 [==============================] - 45s 637ms/step - loss: 0.2418 - accuracy: 0.9089 - val_loss: 0.1865 - val_accuracy: 0.9333
Epoch 5/10
71/71 [==============================] - 44s 624ms/step - loss: 0.2148 - accuracy: 0.9249 - val_loss: 0.1765 - val_accuracy: 0.9387
Epoch 6/10
71/71 [==============================] - 44s 625ms/step - loss: 0.2085 - accuracy: 0.9240 - val_loss: 0.1657 - val_accuracy: 0.9453
Epoch 7/10
71/71 [==============================] - 44s 628ms/step - loss: 0.1911 - accuracy: 0.9316 - val_loss: 0.1562 - val_accuracy: 0.9453

## EfficientNet ADAM

In [11]:
model_3 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_3.build([None, 224, 224, 3])

tb_callback_3 = tf.keras.callbacks.TensorBoard(log_dir='logs/effnet(adam)notunning', histogram_freq=1)

model_3.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

history_3 = model_3.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks = [tb_callback_3])

Epoch 1/10
71/71 [==============================] - 48s 540ms/step - loss: 0.5152 - accuracy: 0.8240 - val_loss: 0.2445 - val_accuracy: 0.9360
Epoch 2/10
71/71 [==============================] - 37s 528ms/step - loss: 0.2677 - accuracy: 0.9160 - val_loss: 0.1951 - val_accuracy: 0.9467
Epoch 3/10
71/71 [==============================] - 37s 520ms/step - loss: 0.2227 - accuracy: 0.9262 - val_loss: 0.1723 - val_accuracy: 0.9480
Epoch 4/10
71/71 [==============================] - 37s 524ms/step - loss: 0.2009 - accuracy: 0.9338 - val_loss: 0.1568 - val_accuracy: 0.9520
Epoch 5/10
71/71 [==============================] - 37s 518ms/step - loss: 0.1882 - accuracy: 0.9369 - val_loss: 0.1605 - val_accuracy: 0.9507
Epoch 6/10
71/71 [==============================] - 38s 536ms/step - loss: 0.1715 - accuracy: 0.9453 - val_loss: 0.1475 - val_accuracy: 0.9480
Epoch 7/10
71/71 [==============================] - 37s 517ms/step - loss: 0.1595 - accuracy: 0.9493 - val_loss: 0.1472 - val_accuracy: 0.9507

## Resnet SGD

In [12]:
model_4 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_4.build([None, 224, 224, 3])

tb_callback_4 = tf.keras.callbacks.TensorBoard(log_dir='logs/resnet(sgd)notunnig', histogram_freq=1)

model_4.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

history_4 = model_4.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_4])

Epoch 1/10
71/71 [==============================] - 53s 655ms/step - loss: 0.5156 - accuracy: 0.8076 - val_loss: 0.2657 - val_accuracy: 0.9107
Epoch 2/10
71/71 [==============================] - 46s 646ms/step - loss: 0.3244 - accuracy: 0.8796 - val_loss: 0.2273 - val_accuracy: 0.9280
Epoch 3/10
71/71 [==============================] - 46s 646ms/step - loss: 0.2938 - accuracy: 0.8964 - val_loss: 0.2098 - val_accuracy: 0.9227
Epoch 4/10
71/71 [==============================] - 46s 651ms/step - loss: 0.2856 - accuracy: 0.9009 - val_loss: 0.2104 - val_accuracy: 0.9280
Epoch 5/10
71/71 [==============================] - 46s 643ms/step - loss: 0.2523 - accuracy: 0.9160 - val_loss: 0.1937 - val_accuracy: 0.9307
Epoch 6/10
71/71 [==============================] - 47s 658ms/step - loss: 0.2352 - accuracy: 0.9138 - val_loss: 0.1946 - val_accuracy: 0.9253
Epoch 7/10
71/71 [==============================] - 46s 647ms/step - loss: 0.2210 - accuracy: 0.9196 - val_loss: 0.1957 - val_accuracy: 0.9280

## EfficientNet SGD

In [13]:
model_5 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_5.build([None, 224, 224, 3])

tb_callback_5 = tf.keras.callbacks.TensorBoard(log_dir='logs/effnet(sgd)notunning', histogram_freq=1)

model_5.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

history_5 = model_5.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks = [tb_callback_5])

Epoch 1/10
71/71 [==============================] - 46s 531ms/step - loss: 0.6909 - accuracy: 0.7524 - val_loss: 0.4224 - val_accuracy: 0.8987
Epoch 2/10
71/71 [==============================] - 37s 521ms/step - loss: 0.4058 - accuracy: 0.8844 - val_loss: 0.3069 - val_accuracy: 0.9227
Epoch 3/10
71/71 [==============================] - 39s 550ms/step - loss: 0.3351 - accuracy: 0.8978 - val_loss: 0.2606 - val_accuracy: 0.9347
Epoch 4/10
71/71 [==============================] - 37s 516ms/step - loss: 0.2921 - accuracy: 0.9111 - val_loss: 0.2355 - val_accuracy: 0.9427
Epoch 5/10
71/71 [==============================] - 37s 517ms/step - loss: 0.2791 - accuracy: 0.9102 - val_loss: 0.2171 - val_accuracy: 0.9400
Epoch 6/10
71/71 [==============================] - 37s 516ms/step - loss: 0.2547 - accuracy: 0.9236 - val_loss: 0.2082 - val_accuracy: 0.9453
Epoch 7/10
71/71 [==============================] - 37s 517ms/step - loss: 0.2475 - accuracy: 0.9244 - val_loss: 0.1976 - val_accuracy: 0.9453

##Висновок. Як можна замітити, найточнішим методом виявився EfficientNet ADAM а найгіршим CNN